In [1]:
# import libraries
from pyopenms import * # main package used for handling MS data
import os # changing directories
import pandas as pd # to read in tsv and for dataframe creation/manipulation
import subprocess # running R script 

import tensorize # for formatting data
import match 
from constants import ION_TYPES, DEFAULT_MAX_CHARGE

import losses # for getting spectral angle

In [2]:
# after running main script
# change experiment files in DeNovo_main.py as well for now
from DeNovo_main import *

Experiment Files: 
HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD30.mzML
App-2022-06-12_22-28-53.log
HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD30_realtimesearch.tsv
-----------------------------------------------------------------------------------------
Number of MS2 scans: 81780
Number of MS3 scans: 681
-----------------------------------------------------------------------------------------
Precursor mzs match up after taking rounding discrepencies into consideration
Fragment mzs match up perfectly!


In [3]:
new_fragment_df

,Scan_Number,Sequence,Charge,Sequence_Length,Missing_Fragment_Locations,Number_Missing,Target_Fragment,MS3_Scan,Fragment_Sequence,Target_Charge,Locations_Found
13,4039,VSLTQKTDPSVRPMHE,4,16,,0,y4,4048,PMHE,3,[2]
40,4610,AGLNVTTSHSPAAPGE,2,16,"1,3,8,15",4,y3,4621,PGE,1,[2]
42,4630,AGLNVTTSHSPAAPGE,2,16,1,1,y3,4639,PGE,1,[2]
55,4986,HASIQMNVAE,2,10,,0,b5,4994,HASIQ,1,[2]
60,5057,AAKLQTTKVKKPTGTRNLYLARE,5,23,,0,b4,5062,AAKL,3,[3]
88,5600,FKPNKPKPCGLCNQFGHE,4,18,,0,y3,5602,GHE,1,[2]
91,5713,GSSIKKAQQAVANKALTE,3,18,,0,b2,5723,GS,1,[1]
133,7286,IVGGATRIPAVKE,3,13,2,1,y2,7299,KE,1,[1]
141,7640,KPAKAITSSRVPGE,3,14,,0,b2,7644,KP,1,[1]
157,8134,RRQLIVPPHLAHGE,4,14,,0,b6,8141,RRQLIV,2,"[4, 5]"


In [4]:
# creating stick plot for single case 

In [5]:
# choose index where length of fragment sequence is greater than 2
# pretrained model can not predict on peptides less than 3 in length
s = specM3[212]

# obtain mz and intensity values 
mz, intensity = s.get_peaks()

In [6]:
# is the target a b or y ion?
# there exists a special case when there is an internal y ion from a b target 
# the pretrained model does not take this into account 
new_fragment_df['Target_Fragment'][212][0]

'y'

In [7]:
# special case when there is an internal y ion in a b fragment
def findTargetMZs(peptide_object, charge, i):
    
    mzs = []
    
    # targeted fragment is a y ion
    if new_fragment_df['Target_Fragment'][i].startswith('y'):
        y_num = new_fragment_df['Target_Fragment'][i][-1]

        # the full sequence of the fragment
        full_seq = peptide_object.getSuffix(int(y_num))

        # checking fragment for its y ions
        for ion in range(1, int(y_num)):
            y_ion = full_seq.getSuffix(ion)
            for z in range(1, charge):
                mz_y = y_ion.getMonoWeight(Residue.ResidueType.YIon, z) / z
                mzs.append(mz_y)

        # checking fragment for b ions
        for ion in range(1, int(y_num)):
            b_ion = full_seq.getPrefix(ion)
            for z in range(1, charge):
                mz_b = b_ion.getMonoWeight(Residue.ResidueType.Internal, z) / z
                mzs.append(mz_b)
                
    # targeted fragment is a b ion
    elif new_fragment_df['Target_Fragment'][i].startswith('b'):
        b_num = new_fragment_df['Target_Fragment'][i][-1]

        # the full sequence of the fragment
        full_seq = peptide_object.getPrefix(int(b_num))

        # checking fragment for its b ions
        for ion in range(1, int(b_num)):
            b_ion = full_seq.getPrefix(ion)
            for z in range(1, charge):
                mz_b = b_ion.getMonoWeight(Residue.ResidueType.BIon, z) / z
                mzs.append(mz_b)

        # checking fragment for y ions
        for ion in range(1, int(b_num)):
            y_ion = full_seq.getSuffix(ion)
            for z in range(1, charge):
                mz_y = y_ion.getMonoWeight(Residue.ResidueType.Internal, z) / z
                mzs.append(mz_y)
    return mzs

In [8]:
# create peptide object for the target sequence
peptide_object = AASequence.fromString(new_fragment_df['Sequence'][212])
    
# the charge associated with this sequence
charge = new_fragment_df['Target_Charge'][212] + 1
    
# call findTargetMZs function 
# returns list of all mzs associated with target sequence
mzs = findTargetMZs(peptide_object, charge, 212)

In [9]:
# want only mzs/intensities that correspond to the sequence of the target frag
mz_new = []
remove_list = []
count = -1
for m in mz:
    count = count + 1
    if m < max(mzs)+0.02 and m > min(mzs)-0.02: # range of interest, based on the target mzs
        mz_new.append(m)
    else:
        remove_list.append(count) # indicies of intensities to remove

intensity_new = np.delete(intensity, remove_list)

# normalize intensities (values between 0 and 1)
base_normalized = [x/(intensity_new.max()) for x in intensity_new]

In [10]:
# save to csv for plotting stick plot for a single case in R
oneScan_mz_intensity = pd.DataFrame()
observed = oneScan_mz_intensity.assign(mz=mz_new, Intensity=base_normalized)
#observed.to_csv('mzs_intensities_one.csv', index=False)

In [11]:
# create dataframe with all mzs and intensities for this experiment
rows = []
for i in new_fragment_df.index:
    # spectrum
    s = specM3[i]
    
    # obtain mz and intensity values 
    mz, intensity = s.get_peaks()
    
    # base normalize intensity 
    intensity_normed = [x/(intensity.max()) for x in intensity]
    
    #mz_mod = " ".join(str(m) for m in mz)
    #intensity_mod = " ".join(str(i) for i in intensity)
    
    targets = new_fragment_df['Target_Fragment'][i]
    
    # create dict (rows of dataframe)
    data = {'mz':mz,
       'intensity':intensity_normed,
           'target_fragment':targets}
    
    rows.append(data)
    
# create dataframe
mz_intens_df = pd.DataFrame(rows)

# save
#df.to_csv('mzs_intensities.csv', index=False)

In [12]:
# for all scans in experiment:
# calculate spectral angle for all pairs of observed vs predicted
# have to get predicted values by running pretrained model on compute cluster

In [13]:
# create csv file used for input for predicted data
# contains modified_sequence, collision_energy, precursor_charge
mod_seqs = new_fragment_df['Fragment_Sequence']
targ_charge = new_fragment_df['Target_Charge']
energy = int(realtime.split('_')[-2][-2:])

# parse collision energy
collision = []
analyzer = []
for i in range(0, len(mod_seqs)):
    collision.append(energy)
    analyzer.append('FTMS')

In [14]:
# create dataframe and convert to csv
peptidelist = pd.DataFrame()
peptidelist =  peptidelist.assign(modified_sequence=mod_seqs, collision_energy=collision, precursor_charge=targ_charge)
peptidelist.reset_index(inplace=True)

# remove all modified sequences
for i in peptidelist.index:
    if ('[' or ']') in peptidelist['modified_sequence'][i]:
        peptidelist.drop(i, axis=0, inplace=True)
        df.drop(i, axis=0, inplace=True)
        analyzer.pop(i)
        
# removing index column before saving (because of resetting index)
peptidelist = peptidelist.drop('index', axis=1)

In [15]:
# save 
#peptidelist.to_csv('peptidelist.csv', index=False) 
# next, use this csv file to run pretrained model and get predictions 

In [16]:
# format observed data for the spectral angle function 
# function wants similar format as mass_pred from tensorize.py

In [36]:
# for predicted values, convert msms to csv file and load in
predicted = pd.read_csv('predicted2.csv', engine='python')

In [37]:
# list of all predicted intensities
pred_intensities = []
for i in predicted.index:
    if str(predicted['Intensities'][i]) != 'nan':
        split_intens = predicted['Intensities'][i].split(';')
        intensities = [float(intens) for intens in split_intens]
    else: 
        intensities = [0]
    pred_intensities.append(intensities)

In [56]:
# arrays need to have similar shapes
length_pred = [len (l) for l in pred_intensities]
length_obs = [len (l) for l in list(mz_intens_df['intensity'])]

if max(length_pred) < max(length_obs):
    length = max(length_pred)
else:
    length = max(length_obs)

21


In [73]:
pred_array = np.zeros([len(list(predicted['Intensities'])), length], dtype=float)

# replace zeros in array with our values
for i in predicted.index:
    values = pred_intensities[i]
    j = -1 # for indexing
    try:
        for v in values:
            j = j + 1
            pred_array[i, j] = v
    except IndexError: # if this data is larger than observed
        print('skip')

skip


In [77]:
obs_array = np.zeros([len(list(mz_intens_df['intensity'])), max(length)], dtype=float)

# replace zeros in array with our values
for i in mz_intens_df.index:
    values = mz_intens_df['intensity'][i]
    j = -1 # for indexing
    #if len(values) < max(lengths):
        #diff = max(lengths) - len(values)
        #for d in range(0, diff+1):
            #obs_array[i, -d] = -1
    try:
        for v in values:
            j = j + 1
            obs_array[i, j] = v
    except IndexError:
        print('skip')

In [79]:
def get_spectral_angle(true, pred, batch_size):
    import tensorflow.compat.v1 as tf
    
    n = true.shape[0]
    sa = numpy.zeros([n])

    def iterate():
        if n > batch_size:
            for i in range(n // batch_size):
                true_sample = true[i * batch_size : (i + 1) * batch_size]
                pred_sample = pred[i * batch_size : (i + 1) * batch_size]
                yield i, true_sample, pred_sample
            i = n // batch_size
            yield i, true[(i) * batch_size :], pred[(i) * batch_size :]
        else:
            yield 0, true, pred
    
    for i, t_b, p_b in iterate():
        tf.compat.v1.reset_default_graph() 
        with  tf.compat.v1.Session() as s:
            sa_graph = losses.masked_spectral_distance(t_b, p_b)
            sa_b = 1 - s.run(sa_graph)
            #print(sa_b)
            sa[i * batch_size : i * batch_size + sa_b.shape[0]] = sa_b
    sa = numpy.nan_to_num(sa)
    return sa

In [80]:
# get spectral angle using array of our data
batch_size = 1
sa = get_spectral_angle(obs_array, pred_array, batch_size)

In [81]:
sa

array([0.3007078 , 0.08159955, 0.5380853 , 0.10453641, 0.5563665 ,
       0.11814745, 0.        , 0.04196039, 0.24083421, 0.38004153,
       0.24209237, 0.05277456, 0.        , 0.22792517, 0.18933355,
       0.12045055, 0.        , 0.        , 0.        , 0.08878557,
       0.10009319, 0.09591923, 0.03653597, 0.30477881, 0.15065963,
       0.12285872, 0.        , 0.52636321, 0.54466414, 0.11185769,
       0.        , 0.33026144, 0.09561613, 0.38479024, 0.15265797,
       0.        , 0.        , 0.09482324, 0.42960649, 0.15518944,
       0.08626047, 0.40360738, 0.10187617, 0.10337836, 0.08733634,
       0.0778764 , 0.62605137, 0.06605381, 0.        , 0.08019339,
       0.12291468, 0.01299778, 0.07553171, 0.0533993 , 0.22947107,
       0.04467586, 0.27008528, 0.10675385, 0.22545915, 0.        ])